# ValidMind for model validation — 114 Finalize testing and reporting

Learn how to use ValidMind for your end-to-end model validation process with our series of four introductory notebooks. In this last notebook, you'll configure and run some custom tests, then add test results and findings to your validation report.

As we concluded in [113 Perform validation tests](113-perform_validation_tests.ipynb), our challenger random forest classification model was not a viable candidate for our use case. We'll finish up by comparing our champion application scorecard model against our remaining challenger logistic regression model, then use the ValidMind Platform to put together our validation report supplemented by our logged test results as evidence and findings.

## Prerequisites

In order to finalize the validation testing and reporting for your sample model, you'll need to first have:

- [ ] Registered a model within the ValidMind Platform and granted yourself access to the model as a validator
- [ ] Installed the ValidMind Library in your local environment, allowing you to access all its features
- [ ] Learned how to import and initialize datasets for use with ValidMind
- [ ] Learned how to enable custom context for test descriptions generated by ValidMind
- [ ] Understood the basics of how to identify and run validation tests
- [ ] Run data quality and model performance tests for your champion and challenger models, and logged the results of those tests to the ValidMind Platform


<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Need help with the above steps?</b></span>
<br></br>
Refer to the first three notebooks in this series:

<ol>
    <li><a href="111-import_champion_model.ipynb" style="color: #DE257E;"><b>111 Import the champion model</b></a></li>
    <li><a href="112-develop_challenger_models.ipynb" style="color: #DE257E;"><b>112 Develop potential challenger models</b></a></li>
    <li><a href="113-perform_validation_tests.ipynb" style="color: #DE257E;"><b>113 Perform validation tests</b></a></li>
</ol>

</div>

## Setting up

## In summary

## Next steps

### Work with your validation report

### Learn more

#### More how-to guides and code samples

#### Discover more learning resources